In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import Dropdown

import sys
sys.path.append("src")

from src.energy_es.data.prices import PricesManager
from src.energy_es.data.tools import get_time

In [2]:
# Price unit
unit_wid = Dropdown(options=[("€/KWh", "k"), ("€/MWh", "m")], value="k")
unit_wid

Dropdown(options=(('€/KWh', 'k'), ('€/MWh', 'm')), value='k')

In [34]:
unit = unit_wid.value
unit_label = unit_wid.label

print("Price unit selected:", unit_label)

Price unit selected: €/KWh


# Get data

In [35]:
pm = PricesManager()
spot = pm.get_spot_market_prices(unit)
pvpc = pm.get_pvpc_prices(unit)

# Samples
print(spot[0])
print(pvpc[0])

{'datetime': datetime.datetime(2022, 12, 13, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'value': 0.0041}
{'datetime': datetime.datetime(2022, 12, 13, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'value': 0.0423}


# Transform data

In [36]:
prices = []

for i in (spot, pvpc):
    i2 = list(map(
        lambda x: {
            "time": get_time(x["datetime"]),
            "value": x["value"]
        },
        i
    ))
    
    prices.append(i2)

spot_2, pvpc_2 = prices

# Samples
print(spot_2[0])
print(pvpc_2[0])

{'time': '00:00', 'value': 0.0041}
{'time': '00:00', 'value': 0.0423}


# Title and source

In [47]:
dt = spot[0]["datetime"]
dt = dt.strftime(f"%A {dt.day} %B %Y (%Z)")

title = f"Electricity price ({unit_label}) in Spain for {dt}"
source = "Data source: Red Eléctrica de España"

print(title)
print(source)

Electricity price (€/KWh) in Spain for Tuesday 13 December 2022 (UTC+01:00)
Data source: Red Eléctrica de España


# Create dataframes

In [49]:
spot_df = pd.DataFrame(spot_2)
pvpc_df = pd.DataFrame(pvpc_2)

text = ["MIN", "MAX"]
text_pos = ["bottom center", "top center"]

for df in (spot_df, pvpc_df):
    cond = [
        df["value"] == df["value"].min(),
        df["value"] == df["value"].max()
    ]
    
    df["text"] = np.select(cond, text, default=None)
    df["text_position"] = np.select(cond, text_pos, default="top center")

# Create chart

In [50]:
fig = go.Figure()

fig.update_layout(
    title={
        "text": f'{title}<br><span style="font-size: 14px">{source}</span>',
        "yref": "paper",
        "y": 1,
        "yanchor": "bottom",
        "pad": {"l": 77, "b": 40},
        "x": 0,
        "xanchor": "left"
    },
    plot_bgcolor="white",
    xaxis={
        "title": "Time",
        "fixedrange": True,
        "showline": True,
        "mirror": True,
        "linecolor": "black",
        "gridcolor": "lightgrey",
        "ticks": "outside",
        "tickangle": 45
    },
    yaxis={
        "title": unit_label,
        "fixedrange": True,
        "showline": True,
        "mirror": True,
        "linecolor": "black",
        "gridcolor": "lightgrey",
        "ticks": "outside"
    }
)

hover_tem = "Time: &nbsp;%{x}<br>Price: &nbsp;%{y} " + unit_label
spot_hover_tem = "<b>Spot Market</b><br>" + hover_tem
pvpc_hover_tem = "<b>PVPC</b><br>" + hover_tem

# Spot market prices
spot_sca = go.Scatter(
    x=spot_df["time"],
    y=spot_df["value"],
    mode="lines+markers+text",
    text=spot_df["text"],
    line={"width": 3, "color": "#2077b4"},
    marker={"size": 12, "color": "#2077b4"},
    textposition=spot_df["text_position"],
    textfont={"color": "#2077b4"},
    name="Spot Market price",
    hovertemplate=spot_hover_tem,
    hoverlabel={"namelength": 0}
)

# PVPC prices
pvpc_sca = go.Scatter(
    x=pvpc_df["time"],
    y=pvpc_df["value"],
    mode="lines+markers+text",
    text=pvpc_df["text"],
    line={"width": 3, "color": "#ff8c00"},
    marker={"size": 12, "color": "#ff8c00"},
    textposition=pvpc_df["text_position"],
    textfont={"color": "#ff8c00"},
    name="PVPC price",
    hovertemplate=pvpc_hover_tem,
    hoverlabel={"namelength": 0}
)

fig.add_traces([pvpc_sca, spot_sca])
conf = {"displayModeBar": False}

fig.show(config=conf)